In [3]:
import itertools
import math
import random

import numpy as np
import pandas as pd
from tqdm import tqdm

In [4]:
tqdm.pandas()

In [8]:
class MatrixGame:
    def __init__(self, payoffs, social_payoff=None) -> None:
        self.number_of_players = payoffs.shape[0]
        self.number_of_actions = payoffs.shape[1:]

        self.payoffs = payoffs
        self.social_payoff = social_payoff if social_payoff is not None else np.sum(self.payoffs, axis=0)

    def pure_nash_equilibria(self):
        nash_equilibria = set()
        for joint_action in itertools.product(*(range(number_of_player_actions) for number_of_player_actions in self.number_of_actions)):
            is_nash_equilibrium = True
            for i in range(self.number_of_players):
                payoff_options = self.payoffs[i][*(joint_action[j] if j != i else slice(None) for j in range(self.number_of_players))]

                if payoff_options[joint_action[i]] < np.max(payoff_options):
                    is_nash_equilibrium = False
                    break

            if is_nash_equilibrium:
                nash_equilibria.add(joint_action)

        return nash_equilibria

    def pareto_optima(self):
        pareto_optimal_actions = set()
        for joint_action in itertools.product(*(range(number_of_player_actions) for number_of_player_actions in self.number_of_actions)):
            is_pareto_optimal = True
            for other_joint_action in itertools.product(*(range(number_of_player_actions) for number_of_player_actions in self.number_of_actions)):
                if joint_action == other_joint_action:
                    break

                if all(self.payoffs[i][joint_action] <= self.payoffs[i][other_joint_action] for i in range(self.number_of_players)) and any(self.payoffs[i][joint_action] < self.payoffs[i][other_joint_action] for i in range(self.number_of_players)):
                    is_pareto_optimal = False
                    break
            
            if is_pareto_optimal:
                pareto_optimal_actions.add(joint_action)

        return pareto_optimal_actions

    def social_optima(self): 
        socially_optimal_value = np.max(self.social_payoff)
    
        return set(joint_action for joint_action in itertools.product(*(range(number_of_player_actions) for number_of_player_actions in self.number_of_actions)) if self.social_payoff[joint_action] == socially_optimal_value)
    
    def payoff(self, player, joint_action):
        return self.payoffs[player][joint_action]

## Test some well-known 2x2 games

In [9]:
# game = MatrixGame(payoffs=np.array([[[1, -1], [-1, 1]], [[-1, 1], [1, -1]]])) # Matching Pennies
# game = MatrixGame(payoffs=np.array([[[2, -1], [-1, 1]], [[1, -1], [-1, 2]]])) # Battle of the Sexes
# game = MatrixGame(payoffs=np.array([[[-1, -3], [0, -2]], [[-1, 0], [-3, -2]]])) # Prisoners' Dilemma
game = MatrixGame(payoffs=np.array([[[0, -1], [1, -10]], [[0, 1], [-1, -10]]])) # Hawk/Dove

In [10]:
print(f'{game.pure_nash_equilibria()=}, {game.social_optima()=}, {game.pareto_optima()=}')

game.pure_nash_equilibria()={(0, 1), (1, 0)}, game.social_optima()={(0, 1), (1, 0), (0, 0)}, game.pareto_optima()={(0, 1), (1, 0), (0, 0)}


## Loop through all 3x3 games

In [11]:
number_of_players = 2
number_of_actions = 3

joint_actions = list(itertools.product(range(number_of_actions), repeat=number_of_players))
number_of_joint_actions = len(joint_actions)

preference_orders = list(itertools.permutations(range(number_of_joint_actions)))

In [12]:
# Sample 1000 preference orders to speed up experimentation
preference_orders = random.sample(preference_orders, 1_000)

In [13]:
def restrict_game(game, forbidden_action):
    payoffs = game.payoffs
    social_payoff = game.social_payoff
    for i in range(game.number_of_players):
        payoffs = np.delete(payoffs, forbidden_action, i + 1)
        social_payoff = np.delete(social_payoff, forbidden_action, i)
    
    return MatrixGame(payoffs, social_payoff)

In [14]:
def replace_actions(joint_action, forbidden_action):
    return tuple(action if action < forbidden_action else action + 1 for action in joint_action)

In [415]:
results, restrictions = [], []
for preference_matrix in tqdm(map(lambda order: np.array(order).reshape([number_of_actions] * number_of_players), preference_orders), total=len(preference_orders)):
    game = MatrixGame(payoffs=np.array([preference_matrix, preference_matrix.T]))

    result = {'p': game.payoffs, 's': game.social_optima(), 'n': game.pure_nash_equilibria()}
    result['so'] = max(game.social_payoff[a] for a in result['s'])
    result['mesu-'] = min((game.social_payoff[a] for a in result['n']), default=-1)
    result['mesu+'] = max((game.social_payoff[a] for a in result['n']), default=-1)

    for forbidden_action in range(number_of_actions):
        restricted_game = restrict_game(game, forbidden_action)

        restriction = result.copy()
        restriction['forbidden_action'] = forbidden_action
        restriction['s-x'] = {replace_actions(joint_action, forbidden_action) for joint_action in restricted_game.social_optima()}
        restriction['n-x'] = {replace_actions(joint_action, forbidden_action) for joint_action in restricted_game.pure_nash_equilibria()}
        restriction['so-x'] = max(game.social_payoff[a] for a in restriction['s-x'])
        restriction['mesu-x-'] = min((game.social_payoff[a] for a in restriction['n-x']), default=-1)
        restriction['mesu-x+'] = max((game.social_payoff[a] for a in restriction['n-x']), default=-1)

        restrictions.append(restriction)

    results.append(result)

results = pd.DataFrame(results)
restrictions = pd.DataFrame(restrictions)

100%|██████████| 1000/1000 [00:00<00:00, 4224.44it/s]


In [416]:
improvements = restrictions[restrictions.apply(lambda row: row['so'] == row['so-x'] and row['mesu-'] >= 0 and row['mesu-'] < row['mesu-x-'], axis=1)]
improvements

,p,s,n,so,mesu-,mesu+,forbidden_action,s-x,n-x,so-x,mesu-x-,mesu-x+
7,"[[[7, 3, 1], [5, 2, 8], [6, 4, 0]], [[7, 5, 6]...","{(0, 0)}","{(1, 2), (2, 1), (0, 0)}",14,12,14,1,"{(0, 0)}","{(0, 0)}",14,14,14
8,"[[[7, 3, 1], [5, 2, 8], [6, 4, 0]], [[7, 5, 6]...","{(0, 0)}","{(1, 2), (2, 1), (0, 0)}",14,12,14,2,"{(0, 0)}","{(0, 0)}",14,14,14
14,"[[[7, 5, 2], [3, 1, 0], [6, 8, 4]], [[7, 3, 6]...","{(0, 0)}","{(2, 2), (0, 0)}",14,8,14,2,"{(0, 0)}","{(0, 0)}",14,14,14
17,"[[[1, 3, 2], [5, 8, 4], [7, 0, 6]], [[1, 5, 7]...","{(1, 1)}","{(1, 1), (2, 2)}",16,12,16,2,"{(1, 1)}","{(1, 1)}",16,16,16
20,"[[[5, 6, 1], [0, 4, 2], [7, 8, 3]], [[5, 0, 7]...","{(1, 2), (2, 1), (0, 0)}","{(2, 2)}",10,6,6,2,"{(0, 0)}","{(0, 0)}",10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...
2980,"[[[7, 2, 6], [1, 0, 8], [4, 3, 5]], [[7, 1, 4]...","{(0, 0)}","{(1, 2), (2, 1), (0, 0)}",14,11,14,1,"{(0, 0)}","{(0, 0)}",14,14,14
2981,"[[[7, 2, 6], [1, 0, 8], [4, 3, 5]], [[7, 1, 4]...","{(0, 0)}","{(1, 2), (2, 1), (0, 0)}",14,11,14,2,"{(0, 0)}","{(0, 0)}",14,14,14
2982,"[[[5, 7, 6], [0, 8, 3], [4, 1, 2]], [[5, 0, 4]...","{(1, 1)}","{(1, 1), (0, 0)}",16,10,16,0,"{(1, 1)}","{(1, 1)}",16,16,16
2988,"[[[6, 5, 8], [3, 0, 2], [1, 4, 7]], [[6, 3, 1]...","{(2, 2)}","{(0, 0)}",14,12,12,0,"{(2, 2)}","{(2, 2)}",14,14,14


In [423]:
improvements.apply(lambda row: (row['mesu-x-'] - row['mesu+']), axis=1).sort_values()

2579   -8
1402   -8
366    -8
2183   -8
799    -6
       ..
614     7
1239    8
309     8
2973    8
708     8
Length: 430, dtype: int64

In [424]:
improvements.loc[708]

p                   [[[3, 5, 7], [2, 1, 6], [0, 8, 4]], [[3, 2, 0]...
s                                                    {(1, 2), (2, 1)}
n                                                            {(0, 0)}
so                                                                 14
mesu-                                                               6
mesu+                                                               6
forbidden_action                                                    0
s-x                                                  {(1, 2), (2, 1)}
n-x                                                  {(1, 2), (2, 1)}
so-x                                                               14
mesu-x-                                                            14
mesu-x+                                                            14
Name: 708, dtype: object

## General n-player, k-action games

In [330]:
number_of_players = 2
number_of_actions = (3, 3)

joint_actions = list(itertools.product(*(range(number_of_player_actions) for number_of_player_actions in number_of_actions)))
number_of_joint_actions = len(joint_actions)

preference_orders = np.array([np.random.permutation(27) for _ in range(1_000)])

game = MatrixGame(payoffs=np.zeros((number_of_players, *number_of_actions)))

games_2 = []
restrictions_2 = []
for preference_matrix in tqdm(map(lambda order: np.array(order).reshape(number_of_actions), preference_orders), total=len(preference_orders)):
    game.payoffs = np.array([preference_matrix, np.swapaxes(preference_matrix, 1, 0), np.swapaxes(preference_matrix, 2, 0)])
    game.social_payoff = np.sum(game.payoffs, axis=0)

    result = {'o': preference_matrix.flatten(), **{joint_action: game.social_payoff[joint_action] for joint_action in joint_actions}, 's': game.social_optima(), 'n': game.pure_nash_equilibria()}
    result['so'] = max(game.social_payoff[a] for a in result['s'])
    result['minesu'] = min((game.social_payoff[a] for a in result['n']), default=-1)
    result['maxesu'] = max((game.social_payoff[a] for a in result['n']), default=-1)

    for forbidden_action in range(3):
        restricted_game = restrict_game(game, forbidden_action)
        result[f's-{forbidden_action}'] = {replace_actions(joint_action, forbidden_action) for joint_action in restricted_game.social_optima()}
        result[f'n-{forbidden_action}'] = {replace_actions(joint_action, forbidden_action) for joint_action in restricted_game.pure_nash_equilibria()}

        result[f'so-{forbidden_action}'] = max(game.social_payoff[a] for a in result[f's-{forbidden_action}'])
        result[f'minesu-{forbidden_action}'] = min((game.social_payoff[a] for a in result[f'n-{forbidden_action}']), default=-1)
        result[f'maxesu-{forbidden_action}'] = max((game.social_payoff[a] for a in result[f'n-{forbidden_action}']), default=-1)

        restrictions_2.append({**{key: result[key] for key in ['o', *joint_actions, 's', 'n', 'so', 'minesu', 'maxesu']}, 
                                'forbidden_action': forbidden_action, 
                                's-x': result[f's-{forbidden_action}'], 
                                'n-x': result[f'n-{forbidden_action}'],
                                'so-x': result[f'so-{forbidden_action}'], 
                                'minesu-x': result[f'minesu-{forbidden_action}'], 
                                'maxesu-x': result[f'maxesu-{forbidden_action}']})

    games_2.append(result)

games_2 = pd.DataFrame(games_2)
restrictions_2 = pd.DataFrame(restrictions_2)

100%|██████████| 1000/1000 [00:00<00:00, 1997.24it/s]


In [331]:
restrictions_2.iloc[:, -11:]

,s,n,so,minesu,maxesu,forbidden_action,s-x,n-x,so-x,minesu-x,maxesu-x
0,"{(1, 1, 2)}","{(0, 0, 1)}",65,54,54,0,"{(1, 1, 2)}","{(1, 2, 2), (2, 1, 2), (2, 2, 1)}",65,40,60
1,"{(1, 1, 2)}","{(0, 0, 1)}",65,54,54,1,"{(2, 0, 2)}","{(0, 2, 0), (2, 0, 2)}",43,23,43
2,"{(1, 1, 2)}","{(0, 0, 1)}",65,54,54,2,"{(0, 1, 0)}","{(0, 0, 1), (1, 1, 1)}",58,27,54
3,"{(2, 0, 2)}","{(0, 0, 0), (2, 0, 2)}",74,69,74,0,"{(2, 2, 2)}","{(2, 2, 2), (1, 1, 2)}",63,41,63
4,"{(2, 0, 2)}","{(0, 0, 0), (2, 0, 2)}",74,69,74,1,"{(2, 0, 2)}","{(0, 0, 0), (2, 0, 2)}",74,69,74
...,...,...,...,...,...,...,...,...,...,...,...
2995,"{(0, 2, 0)}","{(1, 1, 1), (2, 2, 2)}",68,33,60,1,"{(0, 2, 0)}","{(2, 2, 2)}",68,33,33
2996,"{(0, 2, 0)}","{(1, 1, 1), (2, 2, 2)}",68,33,60,2,"{(0, 1, 0)}","{(1, 0, 0), (0, 0, 1), (1, 1, 1), (0, 1, 0)}",63,49,63
2997,"{(0, 1, 2), (0, 1, 0)}","{(0, 0, 1)}",68,62,62,0,"{(1, 1, 2)}","{(2, 2, 2), (2, 1, 1), (1, 2, 1), (1, 1, 2)}",49,19,49
2998,"{(0, 1, 2), (0, 1, 0)}","{(0, 0, 1)}",68,62,62,1,"{(2, 0, 2)}","{(0, 2, 0), (2, 0, 2)}",48,43,48


In [324]:
improvements_2 = restrictions_2[restrictions_2.apply(lambda row: row['so'] == row['so-x'] and row['minesu'] >= 0 and row['minesu'] < row['minesu-x'], axis=1)]
improvements_2

,o,"(0, 0, 0)","(0, 0, 1)","(0, 0, 2)","(0, 1, 0)","(0, 1, 1)","(0, 1, 2)","(0, 2, 0)","(0, 2, 1)","(0, 2, 2)",...,n,so,minesu,maxesu,forbidden_action,s-x,n-x,so-x,minesu-x,maxesu-x
3,"[0, 23, 14, 12, 10, 5, 18, 26, 2, 3, 25, 6, 13...",0,58,46,18,48,20,48,55,33,...,"{(0, 2, 0), (2, 2, 2), (1, 1, 0), (2, 0, 0)}",72,47,72,0,"{(2, 2, 2)}","{(2, 2, 2), (2, 1, 1), (1, 2, 1), (1, 1, 2)}",72,55,72
13,"[7, 15, 25, 12, 1, 8, 20, 26, 19, 22, 2, 24, 9...",21,42,70,56,12,46,62,53,24,...,"{(0, 0, 2), (0, 1, 0)}",70,56,70,1,"{(0, 0, 2)}","{(0, 2, 0), (0, 0, 2), (2, 0, 0)}",70,62,70
17,"[24, 11, 20, 1, 3, 5, 26, 21, 12, 6, 15, 8, 22...",72,23,66,13,40,17,60,60,51,...,"{(0, 0, 2), (2, 2, 2)}",72,54,66,2,"{(0, 0, 0)}","{(0, 0, 0), (1, 1, 0)}",72,59,72
21,"[2, 17, 9, 13, 11, 21, 23, 4, 1, 10, 0, 5, 8, ...",6,47,41,33,19,45,37,50,27,...,"{(0, 0, 2)}",75,41,41,0,"{(2, 2, 2)}","{(1, 2, 1), (1, 1, 2)}",75,48,50
25,"[1, 14, 26, 5, 10, 4, 25, 6, 13, 17, 24, 22, 2...",3,33,77,39,57,47,25,27,49,...,"{(1, 1, 0), (0, 0, 2)}",77,70,77,1,"{(0, 0, 2)}","{(0, 0, 2)}",77,77,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2969,"[7, 14, 17, 25, 20, 8, 21, 13, 11, 6, 18, 2, 1...",21,53,55,37,39,32,41,37,24,...,"{(1, 1, 1), (2, 2, 2)}",78,48,78,2,"{(1, 1, 1)}","{(1, 1, 1)}",78,78,78
2973,"[16, 5, 11, 9, 22, 26, 21, 7, 25, 8, 24, 20, 1...",48,19,43,25,47,61,41,10,48,...,"{(0, 1, 2)}",69,61,61,0,"{(1, 1, 1)}","{(1, 1, 1)}",69,69,69
2984,"[8, 20, 0, 4, 12, 22, 16, 5, 14, 11, 26, 10, 2...",24,44,16,26,59,34,28,48,46,...,"{(2, 1, 2), (1, 1, 0)}",72,61,68,2,"{(1, 1, 1)}","{(1, 1, 0)}",72,68,68
2987,"[23, 22, 11, 19, 0, 4, 7, 2, 16, 6, 15, 25, 20...",69,63,29,31,35,37,33,40,24,...,"{(0, 0, 0), (2, 1, 2), (1, 1, 1), (2, 2, 1)}",72,38,72,2,"{(1, 1, 1)}","{(0, 0, 0), (1, 1, 1)}",72,69,72


In [326]:
improvements_2.apply(lambda row: (row['minesu-x'] - row['minesu']) / row['minesu'], axis=1).describe()

count    485.000000
mean       0.319662
std        0.305341
min        0.014085
25%        0.114754
50%        0.240000
75%        0.415094
max        2.380952
dtype: float64

In [339]:
def generate_payoffs(number_of_players, number_of_actions, number_of_samples):
    return [np.array([np.random.permutation(number_of_actions ** number_of_players).reshape(tuple([number_of_actions] * number_of_players)) for _ in range(number_of_players)]) for _ in range(number_of_samples)]

In [402]:
def generate_symmetric_payoffs(number_of_actions, number_of_samples):
    payoffs = []
    for _ in range(number_of_samples):
        preference_matrix = np.random.permutation(number_of_actions ** number_of_players).reshape(tuple([number_of_actions] * number_of_players))
        payoffs.append(np.array([preference_matrix, preference_matrix.T]))

    return payoffs

In [366]:
number_of_players = 2
number_of_actions = 3
number_of_samples = 1_000

In [367]:
results, restrictions = [], []
for payoffs in generate_payoffs(number_of_players, number_of_actions, number_of_samples):
    game = MatrixGame(payoffs)

    result = {'p': game.payoffs, 's': game.social_optima(), 'n': game.pure_nash_equilibria()}
    result['so'] = max(game.social_payoff[a] for a in result['s'])
    result['mesu-'] = min((game.social_payoff[a] for a in result['n']), default=-1)
    result['mesu+'] = max((game.social_payoff[a] for a in result['n']), default=-1)

    for forbidden_action in range(number_of_actions):
        restricted_game = restrict_game(game, forbidden_action)

        restriction = result.copy()
        restriction['forbidden_action'] = forbidden_action
        restriction['s-x'] = {replace_actions(joint_action, forbidden_action) for joint_action in restricted_game.social_optima()}
        restriction['n-x'] = {replace_actions(joint_action, forbidden_action) for joint_action in restricted_game.pure_nash_equilibria()}
        restriction['so-x'] = max(game.social_payoff[a] for a in restriction['s-x'])
        restriction['mesu-x-'] = min((game.social_payoff[a] for a in restriction['n-x']), default=-1)
        restriction['mesu-x+'] = max((game.social_payoff[a] for a in restriction['n-x']), default=-1)

        restrictions.append(restriction)

    results.append(result)

results = pd.DataFrame(results)
restrictions = pd.DataFrame(restrictions)

In [368]:
restrictions

,p,s,n,so,mesu-,mesu+,forbidden_action,s-x,n-x,so-x,mesu-x-,mesu-x+
0,"[[[7, 4, 1], [8, 5, 2], [3, 0, 6]], [[7, 2, 0]...","{(0, 0)}","{(1, 1)}",14,10,10,0,"{(2, 2)}","{(1, 1)}",12,10,10
1,"[[[7, 4, 1], [8, 5, 2], [3, 0, 6]], [[7, 2, 0]...","{(0, 0)}","{(1, 1)}",14,10,10,1,"{(0, 0)}","{(0, 0), (2, 2)}",14,12,14
2,"[[[7, 4, 1], [8, 5, 2], [3, 0, 6]], [[7, 2, 0]...","{(0, 0)}","{(1, 1)}",14,10,10,2,"{(0, 0)}","{(1, 1)}",14,10,10
3,"[[[3, 5, 4], [8, 2, 6], [7, 0, 1]], [[2, 6, 8]...","{(0, 2)}","{(1, 2)}",12,11,11,0,"{(1, 2)}","{(1, 2)}",11,11,11
4,"[[[3, 5, 4], [8, 2, 6], [7, 0, 1]], [[2, 6, 8]...","{(0, 2)}","{(1, 2)}",12,11,11,1,"{(0, 2)}","{(0, 2)}",12,12,12
...,...,...,...,...,...,...,...,...,...,...,...,...
2995,"[[[8, 5, 3], [1, 4, 2], [0, 7, 6]], [[8, 4, 2]...","{(0, 0)}","{(0, 0)}",16,16,16,1,"{(0, 0)}","{(0, 0)}",16,16,16
2996,"[[[8, 5, 3], [1, 4, 2], [0, 7, 6]], [[8, 4, 2]...","{(0, 0)}","{(0, 0)}",16,16,16,2,"{(0, 0)}","{(0, 0)}",16,16,16
2997,"[[[6, 3, 0], [7, 4, 8], [1, 5, 2]], [[8, 3, 2]...","{(1, 0), (0, 0)}","{(1, 0)}",14,14,14,0,"{(1, 1)}",{},10,-1,-1
2998,"[[[6, 3, 0], [7, 4, 8], [1, 5, 2]], [[8, 3, 2]...","{(1, 0), (0, 0)}","{(1, 0)}",14,14,14,1,"{(0, 0)}","{(0, 0), (2, 2)}",14,7,14


In [369]:
improvements = restrictions[restrictions.apply(lambda row: row['so'] == row['so-x'] and row['mesu-'] >= 0 and row['mesu-'] < row['mesu-x-'], axis=1)]
improvements

,p,s,n,so,mesu-,mesu+,forbidden_action,s-x,n-x,so-x,mesu-x-,mesu-x+
1,"[[[7, 4, 1], [8, 5, 2], [3, 0, 6]], [[7, 2, 0]...","{(0, 0)}","{(1, 1)}",14,10,10,1,"{(0, 0)}","{(0, 0), (2, 2)}",14,12,14
4,"[[[3, 5, 4], [8, 2, 6], [7, 0, 1]], [[2, 6, 8]...","{(0, 2)}","{(1, 2)}",12,11,11,1,"{(0, 2)}","{(0, 2)}",12,12,12
15,"[[[4, 5, 8], [3, 7, 6], [0, 1, 2]], [[5, 1, 3]...","{(1, 1)}","{(1, 1), (0, 0)}",15,9,15,0,"{(1, 1)}","{(1, 1)}",15,15,15
24,"[[[2, 0, 6], [1, 8, 7], [4, 5, 3]], [[7, 8, 0]...","{(1, 1)}","{(1, 1), (2, 0)}",14,8,14,0,"{(1, 1)}","{(1, 1)}",14,14,14
26,"[[[2, 0, 6], [1, 8, 7], [4, 5, 3]], [[7, 8, 0]...","{(1, 1)}","{(1, 1), (2, 0)}",14,8,14,2,"{(1, 1)}","{(1, 1)}",14,14,14
...,...,...,...,...,...,...,...,...,...,...,...,...
2969,"[[[2, 4, 8], [7, 5, 6], [1, 3, 0]], [[4, 0, 6]...","{(1, 0)}","{(1, 0), (0, 2)}",15,14,15,2,"{(1, 0)}","{(1, 0)}",15,15,15
2970,"[[[7, 3, 8], [1, 5, 0], [4, 2, 6]], [[4, 3, 2]...","{(1, 1)}","{(1, 1), (0, 0)}",12,11,12,0,"{(1, 1)}","{(1, 1)}",12,12,12
2973,"[[[0, 8, 2], [1, 7, 4], [3, 5, 6]], [[5, 3, 8]...","{(0, 1), (1, 2)}","{(2, 0)}",11,9,9,0,"{(1, 2)}","{(2, 2)}",11,10,10
2991,"[[[2, 8, 4], [6, 7, 0], [5, 3, 1]], [[0, 5, 8]...","{(1, 1)}","{(0, 2)}",14,12,12,0,"{(1, 1)}","{(1, 1)}",14,14,14


In [374]:
improvements[improvements.apply(lambda row: (row['mesu-x-'] - row['mesu-']) / row['mesu-'], axis=1) >= 1
            ]

,p,s,n,so,mesu-,mesu+,forbidden_action,s-x,n-x,so-x,mesu-x-,mesu-x+
423,"[[[2, 8, 5], [0, 3, 6], [1, 4, 7]], [[3, 1, 0]...","{(2, 2)}","{(2, 2), (0, 0)}",15,5,15,0,"{(2, 2)}","{(2, 2)}",15,15,15
860,"[[[6, 4, 3], [5, 8, 0], [7, 2, 1]], [[0, 2, 4]...","{(1, 1)}","{(1, 1), (0, 2)}",14,7,14,2,"{(1, 1)}","{(1, 1)}",14,14,14
996,"[[[5, 4, 0], [3, 1, 7], [2, 8, 6]], [[2, 1, 0]...","{(1, 2), (2, 2)}","{(1, 2), (0, 0)}",14,7,14,0,"{(1, 2), (2, 2)}","{(1, 2)}",14,14,14
1128,"[[[1, 3, 6], [4, 7, 5], [2, 0, 8]], [[3, 7, 5]...","{(2, 2)}","{(1, 0)}",14,6,6,0,"{(2, 2)}","{(2, 2)}",14,14,14
1410,"[[[5, 3, 6], [1, 2, 8], [7, 0, 4]], [[1, 5, 3]...","{(1, 2)}","{(0, 1), (1, 2), (2, 0)}",16,8,16,0,"{(1, 2)}","{(1, 2)}",16,16,16
1855,"[[[7, 1, 4], [5, 0, 8], [6, 2, 3]], [[7, 0, 6]...","{(0, 0)}","{(2, 1), (0, 0)}",14,7,14,1,"{(0, 0)}","{(0, 0)}",14,14,14
1856,"[[[7, 1, 4], [5, 0, 8], [6, 2, 3]], [[7, 0, 6]...","{(0, 0)}","{(2, 1), (0, 0)}",14,7,14,2,"{(0, 0)}","{(0, 0)}",14,14,14
2706,"[[[4, 5, 3], [2, 7, 1], [6, 8, 0]], [[1, 2, 4]...","{(2, 1)}","{(0, 2), (2, 0)}",15,7,14,0,"{(2, 1)}","{(2, 1)}",15,15,15


In [377]:
improvements.loc[1128].p

array([[[1, 3, 6],
        [4, 7, 5],
        [2, 0, 8]],

       [[3, 7, 5],
        [2, 0, 1],
        [8, 4, 6]]])

In [372]:
improvements.apply(lambda row: (row['so-x'] - row['mesu-x-']) / (row['so'] - row['mesu-']), axis=1).describe()

count    239.000000
mean       0.060171
std        0.179576
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        0.800000
dtype: float64

In [397]:
# g = MatrixGame(np.array([[[-1, 0, 0], [1, 1, -1], [0, -1, 1]], [[-1, 1, 0], [1, -1, 0], [1, -1, 0]]]))

# g = MatrixGame(np.array([[[1, 3, 6], [4, 7, 5], [2, 0, 8]], [[3, 7, 5], [2, 0, 1], [8, 4, 6]]]))

g = MatrixGame(np.array([[[0, 0, 0], [1, 1, 0], [-1, 0, 2]], [[-1, 0, -1], [1, -1, 0], [2, -1, 1]]]))

In [26]:
def show_payoffs(game):
    for player in range(game.number_of_players):
        print(f'Player {player + 1}:\n{game.payoffs[player]}')
    
    print(f'Social payoff:\n{game.social_payoff}')

## Examples

### 3x3 game where restriction achieves the SO

In [32]:
payoff_matrix = np.array([[1, 1, 0], [2, 2, 0], [0, 3, 1]])

game = MatrixGame(np.array([payoff_matrix, payoff_matrix.T]))
print(f'{game.pure_nash_equilibria()=}, {game.social_optima()=}')

restricted_game = restrict_game(game, 2)
print(f'{restricted_game.pure_nash_equilibria()=}, {restricted_game.social_optima()=}')

game.pure_nash_equilibria()={(2, 2)}, game.social_optima()={(1, 1)}
restricted_game.pure_nash_equilibria()={(1, 1)}, restricted_game.social_optima()={(1, 1)}


In [41]:
payoff_matrix = np.array([[1, 1, 0], [2, 2, 3], [0, 6, 4]])

game = MatrixGame(np.array([payoff_matrix, payoff_matrix.T]))
print(f'{game.pure_nash_equilibria()=}, {game.social_optima()=}')

restricted_game = restrict_game(game, 2)
print(f'{restricted_game.pure_nash_equilibria()=}, {restricted_game.social_optima()=}')

game.pure_nash_equilibria()={(2, 2)}, game.social_optima()={(1, 2), (2, 1)}
restricted_game.pure_nash_equilibria()={(1, 1)}, restricted_game.social_optima()={(1, 1)}


In [39]:
show_payoffs(game)

Player 1:
[[1 1 0]
 [0 2 3]
 [0 6 4]]
Player 2:
[[1 0 0]
 [1 2 6]
 [0 3 4]]
Social payoff:
[[2 1 0]
 [1 4 9]
 [0 9 8]]
